In [11]:
import cv2
import numpy as np
import sys
#9x9 Left,Right and consistency check

view1 = cv2.imread('C:/Documents-Summer2018/CVIP/PA2/view1.png',0)
view5 = cv2.imread('C:/Documents-Summer2018/CVIP/PA2/view5.png',0)

disp1 = cv2.imread('C:/Documents-Summer2018/CVIP/PA2/disp1.png',0)
disp5 = cv2.imread('C:/Documents-Summer2018/CVIP/PA2/disp5.png',0)

view1 = np.pad(view1,[(4,4),(4,4)], mode="constant")
view5 = np.pad(view5,[(4,4),(4,4)], mode="constant")
disp1 = np.pad(disp1,[(4,4),(4,4)], mode="constant")
disp5 = np.pad(disp5,[(4,4),(4,4)], mode="constant")


row = int(view1.shape[0])
col = int(view1.shape[1])

disp_1 = np.zeros((row,col));
disp_5 = np.zeros((row,col));

print(view1.shape,view5.shape,disp1.shape,disp5.shape)
print(disp_1.shape,disp_5.shape)








(378, 471) (378, 471) (378, 471) (378, 471)
(378, 471) (378, 471)


In [15]:
#left image disp map
for x in range(4,row-4):     
    for y in range(4,col-4):
        block1 = np.zeros((9,9))
        block1 = view1[x-4:x+5,y-4:y+5] #block for view1
        min_dist = sys.maxsize    #max integer value
        match_y = -1
        for z in range(y-150,y):
            if(z < 5):
                z = 5
            block2 = view5[x-4:x+5,z-4:z+5] #block for view 5
            result = np.sum((block1 - block2) ** 2)                
            if(result < min_dist):
                min_dist = result
                match_y = z               
                
        disp_1[x][y] = np.absolute(y - match_y)

In [16]:
MSE_left = np.mean(np.square(disp1 - disp_1))
print("MSE Left 9x9:",MSE_left)


cv2.imshow('disp1_left_disparity_9x9', disp_1/disp_1.max())
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('C:\Documents-Summer2018\CVIP\PA2\OPTnew\disp1_left_disparity_9x9.jpg',disp_1)

print('disp_1_9x9',disp_1)
print('view1_9x9',view1)

MSE Left 9x9: 564.241521472944
disp_1_9x9 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
view1_9x9 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [19]:
#9x9 Right disparity map
for x in range(4,row-4):     
    for y in range(4,col-4):
        
        block1 = np.zeros((9,9))
        block2 = np.zeros((9,9))
        
        block1 = view5[x-4:x+5,y-4:y+5] #block for view3
        min_dist = sys.maxsize    #max integer value
        for z in range(y,y+100):
            if z > col-5:
                z = col-5
            
            block2 = view1[x-4:x+5,z-4:z+5]
            result = np.sum((block1 - block2) ** 2)                
            if(result < min_dist):
                min_dist = result
                match_y = z               
                
        disp_5[x][y] = np.absolute(y - match_y) 


        
MSE_right = np.square(np.subtract(disp5, disp_5)).mean()#https://stackoverflow.com/questions/16774849/mean-squared-error-in-numpy
print("MSE Right 9x9:",MSE_right)


cv2.imshow('disp5_right_9x9', disp_5/255)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('C:\Documents-Summer2018\CVIP\PA2\OPTnew\disp5_right_disparity_map_9x9.jpg',disp_1)


print('disp_5_9x9',disp_5)
print('view5_9x9',view5)




MSE Right 9x9: 230.02163583055304
disp_5_9x9 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
view5_9x9 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [20]:
#consistency check

consistencyL = np.zeros((row,col))

#fill up consistency left by backprop

for x in range(4,row-4):
    for y in range(4,col-4):
        dispval_left = disp_1[x][y]
        if(y-dispval_left > 0):
            dispval_right = disp_5[x][int(abs(y-dispval_left))]
        else:
            dispval_right = disp_5[x][y]
        if(dispval_left == dispval_right):
            consistencyL[x][y] = dispval_left
        else:                
            consistencyL[x][y] = 0


cv2.imshow('Consistency Left 9x9', consistencyL/255)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('C:\Documents-Summer2018\CVIP\PA2\OPTnew\Consistency_Left_9x9.jpg',consistencyL)
            
mse_cL = np.mean(np.square(disp1 - consistencyL))#https://stackoverflow.com/questions/16774849/mean-squared-error-in-numpy
print("MSE consistency left 9x9:",mse_cL)       



MSE consistency left 9x9: 950.4205956031858


In [21]:
consistencyR = np.zeros((row,col))

for x in range(1,row-1):
    for y in range(1,col-1):
        dispval_right = disp_5[x][y]
        if(y+dispval_right < col-1):
            dispval_left = disp_1[x][int(abs(y+dispval_right))]
        else:
            dispval_left=disp_1[x][y]
        if(dispval_right == dispval_left):
            consistencyR[x][y] = dispval_right
        else:
            consistencyR[x][y] = 0


cv2.imshow('Consistency Right', consistencyR/255)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('C:\Documents-Summer2018\CVIP\PA2\OPTnew\Consistency_Right_9x9.jpg',consistencyR)


mse_cR = np.mean(np.square(disp5 - consistencyR))#https://stackoverflow.com/questions/16774849/mean-squared-error-in-numpy
print("MSE consistency Right 9x9:",mse_cR)

MSE consistency Right 9x9: 828.9149675911884
